In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [54]:
# task3
import os
import json
from langchain_community.chat_models import AzureChatOpenAI
from pprint import pprint

# Set environment variables
os.environ["OPENAI_API_KEY"] = "6ffhCm6wvgxD2LRD7wNZI5sHgqHn4lqYOY7xn8Ycdg7vHvZ8qyujJQQJ99BCACYeBjFXJ3w3AAABACOGwuFA"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"

# Initialize Azure OpenAI GPT model
def initialize_chat_model(temperature=0.0):
    return AzureChatOpenAI(
        deployment_name="gpt-4o",
        azure_endpoint="https://tcoeaiteamgpt4o.openai.azure.com/",
        temperature=temperature
    )



In [55]:

# First prompt to process the input data
def process_input_data(data,Suggestion):
    prompt = f"""

     # Input  :
                    Read the entire file {data} {Suggestion}, it contains the following: [Question-Answer pairs, Domain, Region, Use case, Article laws, Schema with possible_augmentation] listed information, which will be made use for further evaluations.
                    #--------------------------------------------------------------------------------------------------------

                # Instructions for interpreting the input file data :
                    1. Question : The Responsible AI chat application will post the question to the user, to understand about their system.
                    2. Answer : The user provides an answer, describing the details of their application.
                    3. Domain : It tells the domain of the application under test that the user refers to.
                    4. Region : Region/Countries where the system is existent or deployed.
                    5. UseCase : Application functionality is described.
                    6. Schema : Holds all possible augmentations extracted from Knowledge Graph.
                        [possible_augmentation : All the relatable/relevant augmentations to the answer will be present collectively.]

                #--------------------------------------------------------------------------------------------------------

                # System Definitions :
                As an expert in Generative AI and Responsible AI Assurance, analyze the question, answer, domain, region, and use case.
                Focus primarily on the correlation between these elements to select the most relevant augmentations and recommend it for the business analyst to generate more test data.
                Article laws should be considered as a secondary check, not a primary driver of augmentation decisions. They serve as an additional layer of validation but should not drastically impact augmentation selection.
                Also, recommend any additional applicable augmentations beyond those listed in the schema.

                #--------------------------------------------------------------------------------------------------------

                # Steps to Identify the Right Augmentation

                    **importatnt points to remember**
                    1. If the provided answer is related to audio related information (calls , recording , voice note etc.) in that case make sure to consider only the augumentation related to audio based data.
                    2. Similarly If the provided answer is related to image information (screenshot, picture, any image format file ex. png, jpg , jpeg) in that case make sure to consider only the augumentation related to image based data.
                    3. Similarly If the provided answer is related to text information in that case make sure to consider only the augumentation related to image based data.

                    #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


                  1. Correlate the provided details with the augmentations and recommend the most relevant ones. Identify the augmentations based on the [answer] in the input, while considering the [domain], [region], [usecase], and [possible_augmentation] from the [schema].
                  2. Think and reason out why this augmentation will be applicable to this question-answer pair. Provide by a clear justification relevant to the specific question-answer pair.
                  3. If you find that the augmentations do not align with the provided question-answer pair, feel free to propose a new augmentation.Ensure that you carefully think through the reason before suggesting it, and only recommend it if it is relevant to the specific question-answer pair context.
                  5. After generating the full list of augmentations, revisit and refine the augmentation names, descriptions. Ensure they include realistic examples and incorporate terms or keywords from the question-answer pair.Understand the answer provided and add an in-built advice accordingly in the Reason (what to do and what not to do) for better functionality.
                  6. The augmentations should be semantically aligned with the specific question-answer pair, avoiding any generic reasoning.
                  7. The augmentation description should be relevant to the answer, but it’s important not to be overly specific to the example or scenario given in the response.
                  8. Generate example with groundtruth and 2-3 augmented outputs. Example template given below.
                  9. Apply the relevant augmentations and filter the top 5 for each line item. Additionally, ensure that the augmentations are not repeated across the items.
                  10. STRICTLY Ensure that augmentation names should not be metrics name.

                #--------------------------------------------------------------------------------------------------------


                # Examples to be captured :

                    Example 1 :
                        Question: How does the system deal with fluctuating demand during the holidays?
                        Answer: The system automatically adjusts the availability and prioritizes bookings for peak demand periods like Christmas and New Year's.

                        Augmentations can be applied:
                        [
                                  {{
                                    "name": "Shuffle",
                                    "description": "Randomizes the order of words or phrases."
                                    "example :
                                        [
                                            "Original: The development team is working on optimizing the system's performance to reduce downtime,
                                            Augmented: To reduce downtime, the performance system optimizing is working team on development the.
                                            Parameter: Applied sentence reordering and word shuffling to create syntactic variation."
                                        ],
                                      "justification_reason": "Reason for the augmentation."
                                  }},
                                  {{
                                    "name": "Repetition",
                                    "description": "Repeats certain words or phrases to simulate redundancy."
                                    "example :
                                        [
                                            "Original: The system keeps crashing, despite multiple attempts to fix the issue.,
                                            Augmented: The system keeps crashing crashing, despite multiple attempts to fix the issue.
                                            Parameter: Introduced word duplication to simulate repetition-based noise."
                                        ],
                                        "justification_reason": "Reason for the augmentation."
                                  }}
                        ]


                        Justification for Each Augmentation:
                        Rotation: This adjustment allows us to test how the system visually adapts when users or reports require quick interactions under fluctuating demands.
                        Brevity: In a holiday rush, communication needs to be succinct. This ensures the system can maintain clarity while streamlining the response process.
                        Background Noise: During peak seasons, distractions may occur, and this tests the robustness of the system's audio response functionality under realistic conditions.


                #--------------------------------------------------------------------------------------------------------

                # Output Generation :
                    1. Write the Description and Reason in output in such a way that, a wide range of people from different age groups,
                    including younger(10 years to 15 years old) should be able to understand and it should make sense.
                    2. Do not write any other information in the output other than what is expected.
                    Should not be too complicated and confusing.
                    3. Ensure, Reason should not explicitly mention its reading from "the answer", the answer basically tells about the system/application that is being analysed.
                    4. Make sure the augmentation_example should contain orinal ,augment and parameter.

                #--------------------------------------------------------------------------------------------------------




    """

    chat_model = initialize_chat_model()
    response = chat_model.call_as_llm(prompt)
    return response


In [56]:
# Second prompt to process the output of the first response and get final results
def get_final_advisory(data_from_first_response,data,Suggestion):
  # output of first + data + suggetion
    prompt = f"""
            # Input  :
                    Read the entire data {data_from_first_response} and {data}, it contains the following: [Question-Answer pairs, Domain, Region, Use case, Article laws, Schema with possible_augmentation] listed information, which will be made use for further evaluations.
                    #--------------------------------------------------------------------------------------------------------

                # Instructions for interpreting the input file data :
                    1. Question : The Responsible AI chat application will post the question to the user, to understand about their system.
                    2. Answer : The user provides an answer, describing the details of their application.
                    3. Domain : It tells the domain of the application under test that the user refers to.
                    4. Region : Region/Countries where the system is existent or deployed.
                    5. UseCase : Application functionality is described.
                    6. Schema : Holds all possible augmentations extracted from Knowledge Graph.
                        [possible_augmentation : All the relatable/relevant augmentations to the answer will be present collectively.]

                #--------------------------------------------------------------------------------------------------------

                 # Steps to Identify the Right Augmentation

                   a. You are a judge evaluating the provided inputs. Your task is to analyze the given augmentations and mark their applicability
                    as High, Medium or Low  based on two key dimensions:
                    1. From the Suggestion Angle:
                    Review the augmentation name, description, and original augmentation against the {Suggestion} provided.
                    If any of the keywords in these fields strongly resonate with the provided suggestions, mark it as High.
                    If there is a moderate alignment, mark it as Medium.
                    If there is little to no alignment, mark it as Low.
                    2. From the Use Case Judgment Angle:
                    Evaluate if the suggested augmentation is critical for the given Q&A pair and whether it plays a vital role in exposing sensitive bugs during testing.
                    If the augmentation is highly applicable to the use case and necessary for effective test coverage, mark it as High.
                    If it is moderately relevant, mark it as Medium.
                    If it is not relevant, mark it as Low.

                    **Final Applicability Decision Rule:**
                    If either of the two evaluations (Suggestion Angle or Use Case Judgment) results in a High, Medium or Low the final applicability should also be marked as High, Medium or Low.

                 b. **Applicability Reason (`applicability_reason`):** Provide the reason why the augmentation has been assigned the specified
                  applicability level, based on its importance in the context of the system and its potential impact on performance, reliability, or compliance.

                 # Output Generation :
                  1. Do not make any changes in article_laws in output, it should remain exactly same as given in input.
                  2. Make sure the output looks through all the possible_augumentations and  ensure to add all of them to the output.
                  3. Make sure the output for all the questions and answers should be present as from the {data_from_first_response}.
                  4. Make sure the augmentation_example should contain orinal ,augment and parameter.


                   # Output Augmentation Format [JSON] :

                     [
                      {{
                        "question_no": "question number from the input",
                        "question": "The question mentioned",
                        "answer": "The answer mentioned",
                        "article_laws": [
                          {{
                            "law": "Article name",
                            "article": "Article number",
                            "title": "Title of the Article",
                            "description": "Article description",
                            "reason": "Reason why the article is being identified and mapped for the answers"
                          }},
                          {{
                            "law": "Article name",
                            "article": "Article number",
                            "title": "Title of the Article",
                            "description": "Article description",
                            "reason": "Reason why the article is being identified and mapped for the answers"
                          }}
                        ],
                        "augmentations": [
                          {{
                            "augment_name": "The name of the augmentation",
                            "augment_description": "The description of the augmentation based on the answer",
                            "augment_example": "few example for this augmented answers for reference",
                            "justification_reason": "Tell why and how this augmentation is captured based on the inputs (answer)",
                            "applicability": "High/Medium/Low",
                            "applicability_reason": "Reason for the applicability level
                          }},
                          {{
                            "augment_name": "Another augmentation name",
                            "augment_description": "Another augmentation description based on the answer",
                            "augment_example": " few example for this augmented answers for reference",
                            "justification_reason": "Reason for the second augmentation.",
                            "applicability": "High/Medium/Low",
                            "applicability_reason": "Reason for the applicability level
                          }}
                        ]
                      }},
                      {{
                        "question_no": "question number from the input",
                        "question": "Another question mentioned",
                        "answer": "Another answer mentioned",
                        "article_laws": [
                          {{
                            "law": "Article name",
                            "article": "Article number",
                            "title": "Title of the Article",
                            "description": "Article description",
                            "reason": "Reason why the article is being identified and mapped for the answers, fetch from input"
                          }},
                          {{
                            "law": "Second Article name",
                            "article": "Article number",
                            "title": "Title of the Article",
                            "description": "Article description",
                            "reason": "Reason why the article is being identified and mapped for the answers, fetch from input"
                          }}
                        ],
                        "augmentations": [
                          {{
                            "augment_name": "Another augmentation name",
                            "augment_description": "Another augmentation description based on the answer",
                            "augment_example": " few example for this augmented answers for reference",
                            "justification_reason": "Reason for the augmentation.",
                            "applicability": "High/Medium/Low",
                            "applicability_reason": "Reason for the applicability level
                          }}
                        ]
                      }}
                    ]




    """

    chat_model = initialize_chat_model()
    response = chat_model.call_as_llm(prompt)

    return response

In [57]:
# Function to load JSON data from a file
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [58]:
# Main execution
def main():
    # json file path
    json_file_path = '/content/data1.json'

    # Load JSON data
    json_data = load_json_data(json_file_path)
    Suggestion= "Increase focus on entities such as 'Refunds,' 'Flight Status Inquiry,' and 'Cancellation Policy Inquiry,' as these are underrepresented but critical for customer satisfaction. Additionally, ensure balanced representation of 'Customer Sentiment' across positive, neutral, and negative interactions to better understand customer experiences. Expand test data to include more scenarios involving 'Seating Preferences,' 'Baggage Additions,' and 'Upgrade Requests,' as these are common real-world interactions that require detailed handling. This will ensure comprehensive coverage and improve the system's ability to handle diverse customer needs."

    # Process the input data with the first prompt
    initial_response = process_input_data(json_data,Suggestion)
    # print("Initial Response:")
    # pprint(initial_response)

    # Process the output from the first response to get the final advisory
    final_response = get_final_advisory(initial_response, json_data,Suggestion)
    print("\nFinal Response:")
    pprint(final_response)

# Run the script
if __name__ == "__main__":
    main()



Final Response:
('```json\n'
 '[\n'
 '  {\n'
 '    "question_no": 4,\n'
 '    "question": "Can you briefly describe your application\'s '
 'functionality?",\n'
 '    "answer": "The system records travel agency calls, transcribes them, and '
 'uses Generative AI to extract key details, providing structured summaries '
 'for agents and customers.",\n'
 '    "article_laws": [\n'
 '      {\n'
 '        "law": "GDPR",\n'
 '        "article": "Article 5",\n'
 '        "title": "Principles relating to processing of personal data",\n'
 '        "description": "Ensures data processing is lawful, fair, and '
 'transparent, which is critical for recording and transcribing calls.",\n'
 '        "reason": "The system processes personal data (calls and '
 'transcripts), requiring adherence to principles of lawful and transparent '
 'processing."\n'
 '      },\n'
 '      {\n'
 '        "law": "GDPR",\n'
 '        "article": "Article 6",\n'
 '        "title": "Lawfulness of processing",\n'
 '        

In [ ]:
/10.** Add an **`applicability`** field for each augmentation:
- **High:** This augmentation is highly critical, and it is mandatory for the user, tester, or business analyst to monitor it without fail,
 as skipping it could significantly impact performance, user experience, or business decisions.
- **Medium:** This augmentation is important but not critical. It provides valuable insights that can enhance decision-making, but its
absence won’t significantly impact the core functionality or business outcomes.
- **Low:** This augmentation is of minimal importance and is primarily for informational or supplementary purposes. It has little to no
 direct impact on critical functions or business decisions.
/  11. **Applicability Reason (`applicability_reason`):** Provide the reason why the augmentation has been assigned the specified
applicability level, based on its importance in the context of the system and its potential impact on performance, reliability, or compliance.